In [232]:
import pandas as pd, numpy as np
import seaborn as sns, matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from scipy import stats

In [233]:
pd.set_option('display.max_columns', None)

In [234]:
arabica = pd.read_csv('../data/raw/arabica_data_cleaned.csv')
arabica

,Unnamed: 0,Species,Owner,Country.of.Origin,Farm.Name,Lot.Number,Mill,ICO.Number,Company,Altitude,Region,Producer,Number.of.Bags,Bag.Weight,In.Country.Partner,Harvest.Year,Grading.Date,Owner.1,Variety,Processing.Method,Aroma,Flavor,Aftertaste,Acidity,Body,Balance,Uniformity,Clean.Cup,Sweetness,Cupper.Points,Total.Cup.Points,Moisture,Category.One.Defects,Quakers,Color,Category.Two.Defects,Expiration,Certification.Body,Certification.Address,Certification.Contact,unit_of_measurement,altitude_low_meters,altitude_high_meters,altitude_mean_meters
0,1,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,guji-hambela,METAD PLC,300,60 kg,METAD Agricultural Development plc,2014,"April 4th, 2015",metad plc,NaN,Washed / Wet,8.67,8.83,8.67,8.75,8.50,8.42,10.00,10.00,10.00,8.75,90.58,0.12,0,0.0,Green,0,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.00,2200.00,2075.00
1,2,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,guji-hambela,METAD PLC,300,60 kg,METAD Agricultural Development plc,2014,"April 4th, 2015",metad plc,Other,Washed / Wet,8.75,8.67,8.50,8.58,8.42,8.42,10.00,10.00,10.00,8.58,89.92,0.12,0,0.0,Green,1,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.00,2200.00,2075.00
2,3,Arabica,grounds for health admin,Guatemala,"san marcos barrancas ""san cristobal cuch",NaN,NaN,NaN,NaN,1600 - 1800 m,NaN,NaN,5,1,Specialty Coffee Association,NaN,"May 31st, 2010",Grounds for Health Admin,Bourbon,NaN,8.42,8.50,8.42,8.42,8.33,8.42,10.00,10.00,10.00,9.25,89.75,0.00,0,0.0,NaN,0,"May 31st, 2011",Specialty Coffee Association,36d0d00a3724338ba7937c52a378d085f2172daa,0878a7d4b9d35ddbf0fe2ce69a2062cceb45a660,m,1600.00,1800.00,1700.00
3,4,Arabica,yidnekachew dabessa,Ethiopia,yidnekachew dabessa coffee plantation,NaN,wolensu,NaN,yidnekachew debessa coffee plantation,1800-2200,oromia,Yidnekachew Dabessa Coffee Plantation,320,60 kg,METAD Agricultural Development plc,2014,"March 26th, 2015",Yidnekachew Dabessa,NaN,Natural / Dry,8.17,8.58,8.42,8.42,8.50,8.25,10.00,10.00,10.00,8.67,89.00,0.11,0,0.0,Green,2,"March 25th, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1800.00,2200.00,2000.00
4,5,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,guji-hambela,METAD PLC,300,60 kg,METAD Agricultural Development plc,2014,"April 4th, 2015",metad plc,Other,Washed / Wet,8.25,8.50,8.25,8.50,8.42,8.33,10.00,10.00,10.00,8.58,88.83,0.12,0,0.0,Green,2,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.00,2200.00,2075.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1306,1307,Arabica,juan carlos garcia lopez,Mexico,el centenario,NaN,"la esperanza, municipio juchique de ferrer, ve...",1104328663,terra mia,900,juchique de ferrer,JUAN CARLOS GARCÍA LOPEZ,12,1 kg,AMECAFE,2012,"September 17th, 2012",JUAN CARLOS GARCIA LOPEZ,Bourbon,Washed / Wet,7.08,6.83,6.25,7.42,7.25,6.75,10.00,0.00,10.00,6.75,68.33,0.11,0,0.0,None,20,"September 17th, 2013",AMECAFE,59e396ad6e22a1c22b248f958e1da2bd8af85272,0eb4ee5b3f47b20b049548a2fd1e7d4a2b70d0a7,m,900.00,900.00,900.00
1307,1308,Arabica,myriam kaplan-pasternak,Haiti,200 farms,NaN,coeb koperativ ekselsyo basen (350 members),NaN,haiti coffee,~350m,"department d'artibonite , haiti",COEB Koperativ Ekselsyo Basen,1,2 kg,Specialty Coffee Association,2012,"May 24th, 2012",Myriam Kaplan-Pasternak,Typica,Natural / Dry,6.75,6.58,6.42,6.67,7.08,6.67,9.33,6.00,6.00,6.42,67.92,0.14,8,0.0,Blue-Green,16,"May 24th, 2013",

## Reducción de Variables

In [235]:
arabica = arabica[['Country.of.Origin', 'Variety','Processing.Method', 'Moisture', 'Harvest.Year',
                   'Color', 'unit_of_measurement', 'altitude_mean_meters', 'Category.One.Defects', 'Category.Two.Defects',
                   'Total.Cup.Points']]

## Procesado de Variables

### Tratado de Datos Nulos

Puesto que la gran mayoría de datos nulos se encuentran dentro de los mismos registros, considero que la mejor opción sería eliminarlos todos.

In [236]:
arabica = arabica.dropna()

### Otras conversiones

#### País de Origen

In [237]:
otros = arabica['Country.of.Origin'].value_counts() <= 5
for i in range(len(otros.index)):
    if(otros[i]):
        arabica.loc[arabica["Country.of.Origin"] == otros.index[i], "Country.of.Origin"] = "Other"

#### Variedad

In [238]:
otros = arabica['Variety'].value_counts() == 1
for i in range(len(otros.index)):
    if(otros[i]):
        arabica.loc[arabica["Variety"] == otros.index[i], "Variety"] = "Other"

#### Año de Cosecha

In [239]:
year = {'2015/2016' : 2016,
        '2013/2014' : 2014,
        '2017 / 2018' : 2018,
        '2014/2015' : 2015,
        '2011/2012' : 2012,
        '2016 / 2017' : 2017}

arabica['Harvest.Year'] = arabica['Harvest.Year'].replace(year).astype(int)

#### Color

In [240]:
color = {'Green': 'None',
         'Blue' : ['Blue-Green', 'Bluish-Green']}
arabica['Color'] = arabica['Color'].map(lambda x: next((k for k, v in color.items() if x in v), x))

#### Altitud

In [241]:
mask = arabica['unit_of_measurement'].eq('ft')
arabica.loc[mask, ['altitude_mean_meters']] /= 3.281
arabica= arabica.drop(columns= "unit_of_measurement")

In [ ]:
pd.read_csv('../data/raw/arabica_data_cleaned.csv')

In [242]:
# Outliers

# (Dato curioso: el Monte Everest mide 8,849 m)

arabica= arabica[~(arabica["altitude_mean_meters"] > 9000)]

# Procesado

### Encoding

In [243]:
lb = LabelBinarizer()

arabica.Color = lb.fit_transform(arabica.Color)

In [244]:
le = LabelEncoder()

columnas = ["Country.of.Origin", "Variety", "Processing.Method"]

for columna in columnas:
    arabica[columna] = le.fit_transform(arabica[columna])

In [245]:
# Esto es una alternativa al labelencoding
# arabicadum = pd.get_dummies(arabica)